In [30]:
import numpy as np
from scipy.sparse import linalg as la_sparse

In [21]:
n = 1332
sparsity = 10**-1
A = sparsity*np.random.rand(n,n) + np.diag(np.arange(1,n+1))
#A = 0.5*(A+A.T)

nroot = 10

d = A.diagonal();# + 0.1*np.random.rand(n)
idx = d.argsort()[:nroot]
omega = d[idx]
B = np.eye(n)[:,idx]

# Catastrophic 0 eigenvalue failure seems to be associated with non-sparsity of matrix...

In [31]:
la_sparse.gmres(A,np.ones(n))

(array([6.73984797e-01, 3.67284229e-01, 2.58401568e-01, ...,
        5.68325863e-04, 5.38551315e-04, 5.33161445e-04]), 0)

In [15]:
maxit = 100
thresh_vec = 10**-9
tol = 10**-7

def davidson(d,B,omega):
    L = 2*nroot
    Lmax = n//2

    it = 1
    while it <= maxit:

        omega_old = omega.copy()

        L = B.shape[1]

        sigma = np.zeros_like(B)
        for i in range(L):
            sigma[:,i] = np.dot(A,B[:,i])

        G = np.dot(B.T,sigma)

        omega,alpha = np.linalg.eig(G)
        idx = omega.argsort()[:nroot]
        omega = omega[idx]
        alpha = alpha[:,idx]

        X = np.dot(B,alpha)

        XI_VEC = []
        resid_norm = 0
        for j in range(nroot):

            # Calculate residual
            r = np.dot(sigma, alpha[:, j]) - omega[j]*np.squeeze(X[:,j])

            # Apply preconditioner
            r = r/(omega[j] - d[j])
            normr = np.linalg.norm(r)

            print("    Root {}: e = {:>10.12f} de = {:>10.12f} "
          "|r| = {:>10.12f}".format(j+1, np.real(omega[j]), abs(omega[j] - omega_old[j]), normr))

            # Append correction vectors to expand subspace if they are bigger than a certain threshold
            if normr > thresh_vec:
                XI_VEC.append(r)

        # Calculate convergence parameters
        delta_e = np.linalg.norm(omega[:nroot] - omega_old[:nroot])
        resid_norm += normr

        # Check convergence
        if delta_e < tol:
            print('Davidson Converged...')
            print(omega)
            break
        else:
            if L >= Lmax:
                print('Restarting and collapsing...')
                B = np.dot(B, alpha)
                omega = omega_old.copy()
            else:
                if not len(XI_VEC) == 0:
                    XI_VEC,_ = np.linalg.qr(np.asarray(XI_VEC).T)
                    B = np.concatenate((B,XI_VEC),axis=1)
        it += 1


In [16]:
davidson(d,B,omega)

    Root 1: e = 1.001990858613 de = 0.009041030949 |r| = 204.800155373857
    Root 2: e = 2.075574428299 de = 0.005502372094 |r| = 346.091561815067
    Root 3: e = 3.083729992070 de = 0.007501345713 |r| = 254.563112167826
    Root 4: e = 4.077388305181 de = 0.006124778361 |r| = 345.400727823869
    Root 5: e = 5.026306963843 de = 0.003666942828 |r| = 571.963720423835
    Root 6: e = 6.050116120112 de = 0.002321489814 |r| = 934.714029628695
    Root 7: e = 7.098666616534 de = 0.000734057301 |r| = 2939.554101253982
    Root 8: e = 8.070801662070 de = 0.002703419812 |r| = 841.220518412702
    Root 9: e = 9.016472635207 de = 0.007464002004 |r| = 311.954059698393
    Root 10: e = 10.026297938959 de = 0.007832058893 |r| = 299.668715073110
    Root 1: e = 0.998818477006 de = 0.003172381607 |r| = 78.584360622030
    Root 2: e = 2.071658258552 de = 0.003916169748 |r| = 112.084829175140
    Root 3: e = 3.079946068324 de = 0.003783923746 |r| = 89.508171863994
    Root 4: e = 4.072864240555 de = 0

    Root 1: e = 0.995594397936 de = 0.000009986431 |r| = 5.793416764542
    Root 2: e = 2.066713193007 de = 0.000015815514 |r| = 5.325427417621
    Root 3: e = 3.075214349897 de = 0.000007012772 |r| = 6.689332129166
    Root 4: e = 4.066569722589 de = 0.000088334645 |r| = 25.995242289714
    Root 5: e = 5.015516396948 de = 0.000101904081 |r| = 11.570054657958
    Root 6: e = 6.037127358644 de = 0.000104113257 |r| = 20.994107259060
    Root 7: e = 7.087092635341 de = 0.000184213801 |r| = 17.920758962280
    Root 8: e = 8.058453259024 de = 0.000228260756 |r| = 33.516461188301
    Root 9: e = 9.002238333748 de = 0.000125690753 |r| = 39.833059899166
    Root 10: e = 10.011070835717 de = 0.000207633142 |r| = 42.441924736987
    Root 1: e = 0.995600815007 de = 0.000006417070 |r| = 4.788887719136
    Root 2: e = 2.066707743602 de = 0.000005449406 |r| = 4.318168245735
    Root 3: e = 3.075207703076 de = 0.000006646821 |r| = 5.267266641014
    Root 4: e = 4.066509471981 de = 0.000060250608 |r| 

    Root 1: e = 0.995626878452 de = 0.000000559455 |r| = 0.399650242650
    Root 2: e = 2.066774906505 de = 0.000000899818 |r| = 0.369815164657
    Root 3: e = 3.075239302116 de = 0.000000236664 |r| = 0.462465500326
    Root 4: e = 4.066407175886 de = 0.000000990646 |r| = 1.569238680257
    Root 5: e = 5.015374592929 de = 0.000000533376 |r| = 1.166312292647
    Root 6: e = 6.036703916784 de = 0.000000829218 |r| = 1.908575805423
    Root 7: e = 7.086513767290 de = 0.000006106412 |r| = 2.270382336517
    Root 8: e = 8.057092063911 de = 0.000024697256 |r| = 4.906412613162
    Root 9: e = 9.001851471154 de = 0.000009945033 |r| = 7.270931654836
    Root 10: e = 10.010095003499 de = 0.000008080552 |r| = 5.886711121574
    Root 1: e = 0.995627369414 de = 0.000000490962 |r| = 0.300123918094
    Root 2: e = 2.066775099124 de = 0.000000192619 |r| = 0.272478905671
    Root 3: e = 3.075239792030 de = 0.000000489913 |r| = 0.362013417680
    Root 4: e = 4.066406873782 de = 0.000000302104 |r| = 1.124

KeyboardInterrupt: 

In [5]:
E,V = np.linalg.eig(A)
idx = E.argsort()[:nroot]
E = E[idx]
V = V[:,idx]
print(E)

[ 1.00005927  2.0000437   3.00006576  4.00009767  5.00008709  6.00007897
  7.00008194  8.0000574   9.00006104 10.00009905]


In [6]:
for j in range(nroot):
    nm = np.sum( abs(X[:,j] - V[:,j]) )
    print(nm)

NameError: name 'X' is not defined